In [1]:
# Dependencies

import pandas as pd
import numpy as np
import glob
import os
import timeit                                # To check performance
from datetime import datetime, date, time, timedelta
import matplotlib.pyplot as plt

In [2]:
# Import CSV

tic = timeit.default_timer()                            # Monitor performance

ecobici_rides_df = pd.read_csv("dataset_export_files/ecobici_ride_data-oneInHundredv2.csv", index_col=None, header=0)

toc = timeit.default_timer()                            # Monitor performance
print(f'Time (in seconds) to import CSV file: {round(toc - tic, 2)}')

Time (in seconds) to import CSV file: 0.53


In [3]:
ecobici_rides_df_test = ecobici_rides_df.copy()
ecobici_rides_df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707039 entries, 0 to 707038
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Genero_Usuario         707039 non-null  object 
 1   Edad_Usuario           707039 non-null  int64  
 2   Ciclo_Estacion_Retiro  707039 non-null  int64  
 3   Ciclo_Estacion_Arribo  707039 non-null  int64  
 4   Usage_Timestamp        707039 non-null  object 
 5   Duration(Min)          707039 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 32.4+ MB


In [4]:
ecobici_rides_df_test['Usage_Timestamp'] = pd.to_datetime(ecobici_rides_df_test['Usage_Timestamp'])


In [5]:
ecobici_rides_df_test

,Genero_Usuario,Edad_Usuario,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Usage_Timestamp,Duration(Min)
0,M,28,85,85,2010-02-16 12:45:37.427,3.1
1,M,35,78,63,2010-02-19 13:52:30.700,12.1
2,M,34,85,23,2010-02-21 14:30:46.390,26.8
3,M,46,20,17,2010-02-22 16:17:10.470,6.4
4,F,28,74,46,2010-02-23 18:51:57.057,14.9
...,...,...,...,...,...,...
707034,M,27,316,54,2021-12-05 09:29:40.000,13.3
707035,M,49,158,83,2021-12-05 09:24:30.000,10.8
707036,M,33,417,313,2021-12-05 08:55:46.000,12.1
707037,F,23,467,260,2021-12-05 08:54:12.000,33.7


## Routes

### This next section imports and organizes the list of stations to be used in merges below

In [14]:
# Import the list of stations. This will be used as a merge below. 

stations = pd.read_csv("../Estaciones.csv", index_col='ID', header=0)
stations.head()

,Name,Lat,Lon,districtName
ID,,,,
124,124 CLAUDIO BERNARD-DR. LICEAGA,19.422392,-99.150358,Ampliacion Granada
159,159 HUATABAMPO-EJE 1 PTE. AV. CUAUHTÉMOC,19.407517,-99.155373,Ampliacion Granada
241,E241 EJERCITO NAL-JUAN VAZQUEZ DE LA MELLA,19.438620,-99.207580,Ampliacion Granada
243,243 MIGUEL DE CERVANTES SAAVEDRA-LAGO FILT,19.440839,-99.196712,Ampliacion Granada
350,350 JOSE CLEMENTE OROZCO-CORREGGIO,19.384062,-99.181482,Ampliacion Granada


In [15]:
stations_test = stations.copy()
stations_test.head()

,Name,Lat,Lon,districtName
ID,,,,
124,124 CLAUDIO BERNARD-DR. LICEAGA,19.422392,-99.150358,Ampliacion Granada
159,159 HUATABAMPO-EJE 1 PTE. AV. CUAUHTÉMOC,19.407517,-99.155373,Ampliacion Granada
241,E241 EJERCITO NAL-JUAN VAZQUEZ DE LA MELLA,19.438620,-99.207580,Ampliacion Granada
243,243 MIGUEL DE CERVANTES SAAVEDRA-LAGO FILT,19.440839,-99.196712,Ampliacion Granada
350,350 JOSE CLEMENTE OROZCO-CORREGGIO,19.384062,-99.181482,Ampliacion Granada


In [21]:
stations_sorted = stations_test.sort_values(by='ID')
stations_sorted.head(5)

,Name,Lat,Lon,districtName
ID,,,,
1,1 RIO SENA-RIO BALSAS,19.433571,-99.167809,Cuauhtemoc
2,2 RIO GUADALQUIVIR-RIO BALSAS,19.431386,-99.171695,Cuauhtemoc
3,3 REFORMA-INSURGENTES,19.431655,-99.158668,Juarez
4,4 RIO NILO-RIO PANUCO,19.428309,-99.171713,Cuauhtemoc
5,5 RIO PANUCO-RIO TIBER,19.429720,-99.169363,Cuauhtemoc


### Top routes by year

This is how I created a json file for each year. I just repeated it for each year manually, thouth ideally that could be automated with some kind of iteration.

In [19]:
# Helper function that gets used below. It will create a dictionary with the unique routes as keys, 
#   and the total occurencies of that route as a value. 

def uniqueRoutes(yearRides):

    routes = {}

    for index, row in yearRides.iterrows():
        try:
            route = (row['Ciclo_Estacion_Retiro'], row['Ciclo_Estacion_Arribo'])
            if route not in routes: 
                routes[route] = 1
            else:
                routes[route] +=1
        except: pass

    return routes

In [24]:
# Filter the viajes by year 
routes_2016 = ecobici_rides_df_test.loc[ecobici_rides_df_test.Usage_Timestamp.dt.year==2016]

# Get the unique routes and their occurrences by calling the helper function above.  
unique_routes_2016 = uniqueRoutes(routes_2016)

# Sort the unique routes dictionary so that top routes are at beginning of list. 
sorted_unique_routes_2016 = dict(sorted(unique_routes_2016.items(), key=lambda item: item[1], reverse=True))

# Reorganize the dictionary of unique routes into a dataframe. This will make it easier to export to json
#   a few more steps below. 

lst = []
for (key, value) in sorted_unique_routes_2016.items():
   # Check if key is even then add pair to new dictionary
   lst.append([value, key[0], key[1]])

# Then rename column names. 
sorted_unique_routes_2016_df = pd.DataFrame(lst, columns=['Total_Rides',
                                                       'Start_Station_ID',
                                                       'End_Station_ID'])

# MERGE with the list of stations, so that the station data gets inserted into the list of routes. This
#  will help later to plot the routes in Leaflet. 

# Merge data for the Start Station first. 
sorted_unique_routes_2016_df = pd.merge(sorted_unique_routes_2016_df, stations_sorted, left_on=["Start_Station_ID"],
                          right_on=["ID"],
                          how = 'left')

# Merge data for the End Station next.            
sorted_unique_routes_2016_df = pd.merge(sorted_unique_routes_2016_df, stations_sorted, left_on=["End_Station_ID"],
                          right_on=["ID"],
                          how = 'left')

# Drop excess columns. 
# sorted_unique_routes_2016_df.drop(['ID_x', 'ID_y'], inplace=True, axis=1)

# Rename a bunch of columns 
sorted_unique_routes_2016_df = sorted_unique_routes_2016_df.rename(columns={"Name_x": "Start_Station_Name",
                                               'Lat_x' : 'Start_Station_Lat',
                                               'Lon_x' : 'Start_Station_Lon',
                                               'districtName_x' : 'Start_Colonia',
                                               "Name_y": "End_Station_Name",
                                               'Lat_y' : 'End_Station_Lat',
                                               'Lon_y' : 'End_Station_Lon',
                                               'districtName_y' : 'End_Colonia'})

# Send it to JSON
sorted_unique_routes_2016_df.to_json("Routes/sorted_unique_routes_2016.json", orient='index') 
sorted_unique_routes_2016_df.head()

,Total_Rides,Start_Station_ID,End_Station_ID,Start_Station_Name,Start_Station_Lat,Start_Station_Lon,Start_Colonia,End_Station_Name,End_Station_Lat,End_Station_Lon,End_Colonia
0,72,211,217,211 NEWTON-HORACIO,19.433607,-99.188528,Polanco,217 EULER-AV. HORACIO,19.432888,-99.183605,Polanco
1,58,174,183,174 JOAQUÍN GARCIA-IGNACIO MANUEL ALTAMIRANO,19.441110,-99.161640,San Rafael,183 GABINO BARRERA-GUILLERMO PRIETO,19.437670,-99.164020,San Rafael
2,51,174,257,174 JOAQUÍN GARCIA-IGNACIO MANUEL ALTAMIRANO,19.441110,-99.161640,San Rafael,E257 MANUEL MARIA CONTRERAS-VILLALONGIN,19.433800,-99.166220,Cuauhtemoc
3,48,183,174,183 GABINO BARRERA-GUILLERMO PRIETO,19.437670,-99.164020,San Rafael,174 JOAQUÍN GARCIA-IGNACIO MANUEL ALTAMIRANO,19.441110,-99.161640,San Rafael
4,47,217,211,217 EULER-AV. HORACIO,19.432888,-99.183605,Polanco,211 NEWTON-HORACIO,19.433607,-99.188528,Polanco
